<img src="https://hakin9.org/wp-content/uploads/2019/08/connect-a-flask-app-to-a-mysql-database-with-sqlalchemy-and-pymysql.jpg" width=200>


In [10]:
# Installs sqlalchemy.
!pip install sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#**Q1.** Create a sqlite database file using `create_engine` from `sqlalchemy`  (2 pts)
 1. Use the `os.path` library to check if the database file already exists. Delete the previously made databae before creating a new one. 
 2. Create a sqlite database, the metadata, and the session using sqlalchemy.

In [11]:
from sqlalchemy import create_engine, select, MetaData, Column, Integer, String
from sqlalchemy.orm import sessionmaker
import os

# Deletes the database file if it already exists.
db_path = 'datafile.db'
if os.path.exists(db_path):
  os.remove(db_path)

# Creates a sqlite database, the metadata, and the session using sqlalchemy.
engine = create_engine('sqlite:///%s' % db_path)
metadata = MetaData(engine)
Session = sessionmaker(bind=engine)
session = Session()
metadata.create_all(engine)

#**Q2**. Based on the database schema below, create the tables in the database that you created in Q1. (3 pts)

**Do not use the `Table` class to create the table**. Define three classes that inherit the `declarative_base` class to create the tables and map the classes to the tables. 

Here is the entity relationship diagram for the database:

<img src="https://drive.google.com/uc?export=view&id=1AjTTQbGO2oMT9M8awZHIDGi6_nhNMIsg" width=600>

In [12]:
# Imports the declarative base library and instantiates the declarative base class.
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [13]:
from sqlalchemy import ForeignKey

# Defines 3 classes that inherit the declarative_base class and map to the tables and define the column information.

# Defines the Course class.
class Course(Base):
  __tablename__ = 'course'
  course_id = Column(Integer, primary_key=True)
  name = Column(String)
  desc = Column(String)

# Defines the Student class.
class Student(Base):
  __tablename__ = 'student'
  student_id = Column(Integer, primary_key=True)
  firstname = Column(String)
  lastname = Column(String)

# Defines the Enrollment class.
class Enrollment(Base):
  __tablename__ = 'enrollment'
  id = Column(Integer, primary_key=True)
  term = Column(String)
  year = Column(Integer)
  course_id = Column(Integer, ForeignKey('course.course_id'))
  student_id = Column(Integer, ForeignKey('student.student_id'))

Base.metadata.create_all(engine)

#**Q3.** Load data from csv files into the sqlite tables. (1 pts)
- The csv files are given to you in the code cell below. 

In [14]:
import pandas as pd

# Loads the students data into a pandas dataframe.
student_csv = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/students.csv")
# Inserts the data from the dataframe into the student table.
student_csv.to_sql('student', engine, if_exists='append', index=False)

# Loads the course data into a pandas dataframe.
course_csv = pd.read_csv("https://raw.githubusercontent.com/csbfx/advpy122-data/master/course.csv")
# Inserts the data from the dataframe into the course table.
course_csv.to_sql('course', engine, if_exists='append', index=False)

# Loads the enrollment data into a pandas dataframe.
enrollment_csv = pd.read_csv("https://raw.githubusercontent.com/csbfx/advpy122-data/master/enroll.csv")
# Inserts the data from the dataframe into the enrollment table.
enrollment_csv.to_sql('enrollment', engine, if_exists='append', index=False)

#**Q4**. Query the tables and print the results by accessing the data through the class attributes (2 pt)

In [15]:
# Queries all from the student table and displays the data for each field in each record in the table.
results = session.query(Student).all()
for student in results:
    print(student.student_id, student.firstname, student.lastname)

1013832 Martin Jones
1284729 Jane Brown


In [16]:
# Queries all from the course table and displays the data for each field in each record in the table.
results = session.query(Course).all()
for course in results:
    print(course.course_id, course.name, course.desc)

24852 CS22B Intro to Python II
28732 CS122 Advanced Python
37473 CS22A Intro to Python I


In [17]:
# Queries all from the enrollment table and displays the data for each field in each record in the table.
results = session.query(Enrollment).all()
for enrollment in results:
    print(enrollment.id, enrollment.term, enrollment.year, enrollment.course_id, enrollment.student_id)

1 Fall 2020 24852 1284729
2 Spring 2021 28732 1013832


#**Q5.** What classes are the students with a lastname 'Jones' taking? (2 pts)
- Print the students' firstname, lastname, and the course name from the query results.  

In [18]:
# Queries the student and course table and joins the enrollment table to find out which class does a student with a lastname 'Jones' enroll in.
# Prints the students' firstname, lastname, and the course name from the query results.
s = session.query(Student.firstname, Student.lastname, Course.name).join(Enrollment, Enrollment.student_id == Student.student_id).join(Course, Enrollment.course_id == Course.course_id).filter(Student.lastname == 'Jones').all()
print(s)

[('Martin', 'Jones', 'CS122')]
